<a href="https://colab.research.google.com/github/Arunabh13prt/Music-Genre-Classification-and-Recommendation-System/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import math
import librosa
import numpy as np
from google.colab import drive

drive.mount('/content/drive')


DATASET_PATH = "/content/drive/My Drive/GTZAN/genres_original"
JSON_PATH = "/content/drive/My Drive/mfcc.json"
SAMPLE_RATE = 22050
DURATION = 30  # in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

# Data augmentation
def time_stretch(signal, rate=1.0):
    return librosa.effects.time_stretch(signal, rate=rate)

def pitch_shift(signal, sr, n_steps):
    return librosa.effects.pitch_shift(signal, sr=sr, n_steps=n_steps)

def add_noise(signal, noise_factor=0.005):
    noise = np.random.randn(len(signal))
    augmented_signal = signal + noise_factor * noise
    return augmented_signal

# Extracting MFCCs
def extract_mfccs(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=10):
    data = {
        "mapping": [],
        "mfcc": [],
        "labels": [],
        "paths": []
    }

    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)

    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        if dirpath != dataset_path:
            dirpath_components = dirpath.split("/")
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            for f in filenames:
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

                for d in range(num_segments):
                    start = num_samples_per_segment * d
                    finish = start + num_samples_per_segment

                    mfcc = librosa.feature.mfcc(y=signal[start:finish], sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)

                    if len(mfcc[0]) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.T.tolist())
                        data["labels"].append(i-1)
                        data["paths"].append(file_path)

                    # Applying augmentations
                    signal_augmented = time_stretch(signal[start:finish], rate=1.2)
                    mfcc_augmented = librosa.feature.mfcc(y=signal_augmented, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
                    if len(mfcc_augmented[0]) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc_augmented.T.tolist())
                        data["labels"].append(i-1)
                        data["paths"].append(file_path)

                    signal_augmented = pitch_shift(signal[start:finish], sr=sr, n_steps=4)
                    mfcc_augmented = librosa.feature.mfcc(y=signal_augmented, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
                    if len(mfcc_augmented[0]) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc_augmented.T.tolist())
                        data["labels"].append(i-1)
                        data["paths"].append(file_path)

                    signal_augmented = add_noise(signal[start:finish])
                    mfcc_augmented = librosa.feature.mfcc(y=signal_augmented, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
                    if len(mfcc_augmented[0]) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc_augmented.T.tolist())
                        data["labels"].append(i-1)
                        data["paths"].append(file_path)

    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

# Run the preprocessing
extract_mfccs(DATASET_PATH, JSON_PATH)


Mounted at /content/drive

Processing: blues

Processing: hiphop

Processing: disco

Processing: rock

Processing: jazz

Processing: reggae

Processing: metal

Processing: classical

Processing: pop

Processing: country
